# Coursera Capstone Project on Canada Postal Code cleanup
# Below are the steps used in creating the dataframe
    1. Install wikipedia package
    2. Extract the table from wikipedia page using wp.page()
    3. Convert the html page to store in csv file
    4. Removing the postcode that has 'not assigned' "Borough"
    5. Grouping the cells that has same "Postcode" and then combine the "Neighbourhood" column seperated by comma.
    6. If a cell has a ""Borough"" but a 'Not assigned' "Neighborhood", then the "Neighborhood" will be the same as the "Borough"
    7. Load Geospatial Dataframe
    8. Combine geopspatial dataframe with canada_post dataframe

In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
#!conda install -c conda-forge wikipedia --yes
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import wikipedia as wp
print('Libraries imported.')

Libraries imported.


In [12]:
html = wp.page("List of postal codes of Canada: M").html().encode("UTF-8")
df = pd.read_html(html)[0]
df.to_csv('Canada_postalcode.csv',header=0,index=False)
df.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [13]:
#REmoving Not assigned Borough
df_new=df[df['Borough']!='Not assigned'].reset_index(drop=True)
df.head(5)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [14]:
df_new['Neighbourhood']=df_new.groupby('Postcode')['Neighbourhood'].transform(lambda x: ','.join(x))
df_new.drop_duplicates(inplace=True)
df_new.sort_values(by='Postcode',inplace=True)
df_new.reset_index(drop=True,inplace=True)
df_new.head(5)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
#df.loc[[df['Neighbourhood']=='Not assigned','Neighbourhood']]=df['Borough'][df['Neighbourhood']=='Not assigned']
values = df_new['Borough']
df_new['Neighbourhood'] = values.where(df_new['Neighbourhood'] == 'Not assigned', other=df_new['Neighbourhood'])
df_new.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [17]:
df_new.shape

(103, 3)

In [18]:
!wget -q -O "Geospatial_dat.csv" http://cocl.us/Geospatial_data
print("Data downloaded")

Data downloaded


In [22]:
geospatial_data=pd.read_csv("Geospatial_dat.csv")
geospatial_data.columns=['Postcode','Latitude','Longitude']
geospatial_data.head(5)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [26]:
df_merge=pd.merge(df_new,geospatial_data,on='Postcode',how='left')
df_merge.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848
